In [1]:
import pandas as pd
from collections import Counter

In [2]:
df = pd.read_csv("sub_infer_data.csv")

In [3]:
count_dict = dict(Counter(df["sent_num"]))

In [4]:
df = df.rename(columns={"0":"num", "1":"script"})

In [5]:
df["ratio"] = df["num"].map(lambda x: 1-(x%1000)/count_dict[x//1000])

In [10]:
alpha = 0.95
df["new_score"] = df.apply(lambda x: (1-alpha)*x["score"] + alpha*x["ratio"], axis=1)
df["rank_{}".format(alpha)] = df.groupby("sent_num")["new_score"].rank(method="min", ascending=False)

In [11]:
df.head()

,num,script,2,score,sent_num,rank,ratio,new_score,rank_0.9,rank_0.95
0,500733466000,▲ 석문간척지 임차법인협의회가 한국농어촌공사 당진지사 앞에 공공비축벼 320t을 쌓...,500733466000,-2.124493,500733466,9.0,1.000000,0.843775,3.0,2.0
1,500733466001,석문간척지 임차법인협의회(이하 간척지협의회)가 농림축산식품부의 부당한 간척지 임대료...,500733466001,0.780135,500733466,1.0,0.888889,0.883451,1.0,1.0
2,500733466002,43개 영농조합법인이 소속된 간척지협의회는 이번 벼 야적 시위를 통해 현재 1kg당...,500733466002,0.285188,500733466,2.0,0.777778,0.753148,2.0,3.0
3,500733466003,이들은 지난 12월 7일 농림축산식품부에 탄원서를 제출했지만 “임대료 인하는 올해 ...,500733466003,-1.240078,500733466,6.0,0.666667,0.571329,4.0,4.0
4,500733466004,게다가 임차법인들의 계약기간이 올해 만료되기 때문에 임대료를 인하해도 지난 2년 동...,500733466004,-1.468759,500733466,8.0,0.555556,0.454340,6.0,5.0


In [12]:
sent_list = list(set(df["sent_num"]))
result_dict = {}
col = "rank_{}".format(alpha)

for sent in sent_list:
    tmp = df[df["sent_num"]==sent]
    answer = tmp[tmp[col]==1]["script"].item() + "\n" + tmp[tmp[col]==2]["script"].item() + "\n" + tmp[tmp[col]==3]["script"].item()
    
    result_dict[sent] = answer

/home/miyoun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: `item` has been deprecated and will be removed in a future version
  import sys


In [13]:
sub_fi = pd.read_csv("/home/miyoun/jupyter_sangwon.joo/dacon/KoBERT/dacon_ext_summary/data/extractive_sample_submission_v2.csv")
sub_fi["summary"] = sub_fi["id"].map(lambda x: result_dict[int(x)])
sub_fi.to_csv("/data/sangwon.joo/order_weight_{}.csv".format(alpha), index=False)